In [1]:
import pandas as pd
from numpy import random
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.models import load_model

Using TensorFlow backend.


In [2]:
data = pd.read_csv(r"E:\EEG DATA\train_ordered\02_tcp_le\final\meta_final.csv")
data.head()

,main_index,secondary_index,tse_location,edf_location,start_time,stop_time,label
0,0,0,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0.0,336.000,bckg
1,1,0,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0.0,352.000,bckg
2,2,0,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0.0,306.000,bckg
3,3,0,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0.0,275.000,bckg
4,4,0,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0.0,1273.452,bckg


In [3]:
(data['stop_time'] - data['start_time']).idxmax()

231

In [4]:
maxlen = 231*250

In [6]:
#Folder to load the data from
folder = "E:\\EEG DATA\\train_ordered\\02_tcp_le\\train_ready\\"
features = []

#Making a feature matrix - Links of all CSVs to be loaded
for i in range(509):
        features.append(folder+str(i)+".csv")

In [7]:
#bianrizing the labels
def binarize(label):
    if label == 'seiz':
        return 1
    else:
        return 0
    
data['label'] = data['label'].apply(lambda lab: binarize(lab))

In [75]:
def load_and_process(link):
    #Read data into a dataframe
    data = pd.read_csv(link, header=None)
    #Convert dataframe to numpy matrix for padding
    data = data.values
    #Pad sequences as per maxlen
    data = keras.preprocessing.sequence.pad_sequences(data, padding='post', maxlen=maxlen)
    return data 

#Making a generator 
def generator(features, labels):
    #Pick a random index
    index= random.choice(len(features),1)
    #Load and preprocess the data for that index
    batch_features = load_and_process(features[index[0]])
    batch_features = batch_features.reshape((1, 32, maxlen))
    #Get label for index
    batch_labels = labels[index[0]]
    #YEILD TO THE OVERLORDS!
    yield ( batch_features, batch_labels )

In [62]:
#Visualizing what the generator is actually returning
N = generator(features, data['label'])
print(*N, sep='\n')

KeyboardInterrupt: 

In [76]:
model = Sequential()

#model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(500, input_shape=(32, maxlen)))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 500)               116502000 
_________________________________________________________________
dense_20 (Dense)             (None, 10)                5010      
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 116,507,021
Trainable params: 116,507,021
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit_generator(generator(features, data['label']), samples_per_epoch=1, nb_epoch=3)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=1, epochs=3)`
  """Entry point for launching an IPython kernel.


Epoch 1/3


In [ ]:
model.save('E:\\EEG DATA\\train_ordered\\02_tcp_le\\model1.h5')

In [65]:
model.layers[0].input_shape

(None, 32, 57750)

In [66]:
model.layers[0].output_shape

(None, 500)

In [ ]:
model.layers[1].output_shape